In [ ]:
import os
import re
files =  os.listdir()
pdf_file = next(x for x in files if x[-3:] =='pdf')

In [ ]:
import pdfplumber

with pdfplumber.open(pdf_file) as pdf:
    first_page = pdf.pages[0]
    print(first_page.chars[0])

In [ ]:
households = []
texts_flattened = []
household_id = 0
for page_i, page in enumerate(pdf.pages):
    page_households = []
    household = []
    page_cont=True
    for i, word in enumerate(page.extract_words()):
        if word['x0']==31.75336715694298:
            page_households.append(household)
            household=[]
            word['type']='last_name'
            household.append(word)
        elif not word['text']=='©' and page_cont == True:
            household.append(word)
        else:
            page_cont = False
            pass
    page_households.append(household)

    del page_households[0]
    
    for i, household in enumerate(page_households):
        for ii, text in enumerate(household):
            page_households[i][ii]['id']= f'{page_i}-{i}'
            page_households[i][ii]['page_id']=page_i
            page_households[i][ii]['household_id']=household_id
            household[ii]['localx0']=text['x0']-household[0]['x0']
            household[ii]['localx1']=text['x1']-household[0]['x1']
            household[ii]['localtop']=text['top']-household[0]['top']
            household[ii]['localbottom']=text['bottom']-household[0]['bottom']
            char=list(filter(lambda char: char['x0'] == text['x0'] and char['top']==text['top'], page.chars))[0]
            household[ii]['fontname']=char['fontname']
            household[ii]['size']=char['size']
            texts_flattened.append(household[ii])
        household_id+=1
    households.extend(page_households)

In [ ]:
class household:
    def __init__(self,items_list):
        self.items_list = items_list
        self.name_list=[]
        self.address_list=[]
        self.feature_loop()
        self.find_lat_lon()
        self.split_address()

    def within_address_box(self, text_dict):
        localx0 = 185.66274539392109,
        localtop = -1.6303696435678603,
        localx1 = 250,
        localbottom = 32.10517445703567
        if text_dict['localx0']>= 185.66274539392109 \
            and text_dict['localtop'] >=-1.6304\
            and text_dict['localx1']<=250\
            and text_dict['localbottom'] <= 32.10517445703567:
            self.address_list.append(text_dict['text'])
            return True
        else:
            return False
    
    def within_name_box(self, text_dict):
        localx0 = 185.66274539392109,
        localtop = -1.6303696435678603,
        localx1 = 250,
        localbottom = 32.10517445703567
        if text_dict['localx0']>= 0 \
            and text_dict['localtop'] >=0\
            and text_dict['localx1']<=180\
            and text_dict['localbottom'] <= 0:
            self.name_list.append(text_dict['text'])
            return True
        else:
            return False
    
    def feature_loop(self):
        for item in self.items_list:
            self.within_address_box(item)
            self.within_name_box(item)
        self.address =' '.join(self.address_list)
        self.name =' '.join(self.name_list)
    
    def find_lat_lon(self):
        self.lat_pattern = '[0-9]{2}[.][0-9]{1,6}'
        self.lon_pattern = '[-][0-9][0-9][.][0-9]{1,6}'
        self.lat = re.findall(self.lat_pattern,self.address)
        self.lon = re.findall(self.lon_pattern,self.address)
        if self.lat:
            self.lat = float(self.lat[0])
        else:
            self.lat = None
        if self.lon:
            self.lon = float(self.lon[0])
        else:
            self.lon=None
            
    def split_address(self):
        self.lat_pattern = '[0-9]{2}[.][0-9]{1,6}'
        self.address = re.split(self.lat_pattern,self.address)[0]
    
    def dump(self):
        self.out = {
            'name':self.name,
            'address':self.address,
            'lat':self.lat,
            'lon':self.lon
        }
        return self.out

In [ ]:
out = []
for hh in households:
    house=household(hh)
    out.append(house.dump())

In [ ]:
import pandas as pd
df = pd.DataFrame(out)
df['cagegory']=0

In [ ]:
df.to_csv('needs_categorization.csv',index=False)
df.to_clipboard(index=False)

In [ ]:
df = pd.read_csv('with_categorization.csv',usecols=['name', 'address', 'lat', 'lon', 'category'])

In [ ]:
df_input=pd.read_clipboard()

In [ ]:
df_input

In [ ]:
def make_assignements(df_param, included_categories): 
    df_=df_param.copy()
    X=df_[(df_['lat']>0) & (df_['category'].isin(included_categories))][['lat','lon']].values
    mapper=df_[(df_['lat']>0) & (df_['category'].isin(included_categories))][['lat','lon']].index
    print(len(X))
    from sklearn.cluster import KMeans
    import numpy as np

    kmeans = KMeans(n_clusters=13, random_state=0).fit(X)
    kmeans.labels_

    Y =kmeans.cluster_centers_
    
    from h3 import point_dist
    distances = []
    for y in Y:
        out=[]
        for x in X:
            out.append(point_dist((y),(x),unit='km'))
        out = pd.Series(out,mapper)
        distances.append(out)
        
    out = []
    out_pairs=[]
    while len(distances[-1])>0:
        first=[]
        for i,district in enumerate(distances):
            if not district.empty:
                current_min = district.idxmin()
                first.append(current_min)
                df_.loc[current_min,'district']=i
                for ii, value in enumerate(distances):
                    distances[ii].drop(current_min,inplace=True)
            else:
                pass
        #print(first)
        out.append(first)
    assignments = pd.DataFrame(out)
    
    return df_, assignments

In [ ]:
primary_df, primary_assignments = make_assignements(df_input,[1])
primary_df.to_clipboard(index=False)


In [ ]:
prim_sec_df, prim_sec_assignments= make_assignements(df_input,[1,2])
prim_sec_df.to_clipboard(index=False)

In [ ]:
df.to_clipboard(index=False)

In [ ]:
included_categories = [1,2]
X=df[(df['lat']>0) & (df['category'].isin(included_categories))][['lat','lon']].values
mapper=df[(df['lat']>0) & (df['category'].isin(included_categories))][['lat','lon']].index
print(len(X))
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=13, random_state=0).fit(X)
kmeans.labels_

Y =kmeans.cluster_centers_

from h3 import point_dist
distances = []
for y in Y:
    out=[]
    for x in X:
        out.append(point_dist((y),(x),unit='km'))
    out = pd.Series(out,mapper)
    distances.append(out)

out = []
out_pairs=[]
while len(distances[-1])>0:
    first=[]
    for i,district in enumerate(distances):
        if not district.empty:
            current_min = district.idxmin()
            first.append(current_min)
            df.loc[current_min,'district']=i
            for ii, value in enumerate(distances):
                distances[ii].drop(current_min,inplace=True)
        else:
            pass
    #print(first)
    out.append(first)
assignments = pd.DataFrame(out)